# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 8:32AM,245593,10,0085958736,ISDIN Corporation,Released
1,Sep 2 2022 8:32AM,245593,10,0085958744,ISDIN Corporation,Released
2,Sep 2 2022 8:32AM,245593,10,0085958746,ISDIN Corporation,Released
3,Sep 2 2022 8:32AM,245593,10,0085958745,ISDIN Corporation,Released
4,Sep 2 2022 8:32AM,245593,10,0085958767,ISDIN Corporation,Released
5,Sep 2 2022 8:32AM,245593,10,0085958768,ISDIN Corporation,Released
6,Sep 2 2022 8:32AM,245593,10,0085958773,ISDIN Corporation,Released
7,Sep 2 2022 8:32AM,245593,10,0085958775,ISDIN Corporation,Released
8,Sep 2 2022 8:32AM,245593,10,0085958778,ISDIN Corporation,Released
9,Sep 2 2022 8:32AM,245593,10,0085958781,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,245588,Released,16
39,245590,Released,1
40,245591,Released,3
41,245592,Released,30
42,245593,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245588,NaN,NaN,16.0
245590,NaN,NaN,1.0
245591,NaN,NaN,3.0
245592,NaN,NaN,30.0
245593,NaN,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,1.0,6.0,3.0
245455,3.0,2.0,1.0
245478,26.0,0.0,15.0
245525,0.0,0.0,1.0
245527,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,1,6,3
245455,3,2,1
245478,26,0,15
245525,0,0,1
245527,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,1,6,3
1,245455,3,2,1
2,245478,26,0,15
3,245525,0,0,1
4,245527,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,1,6,3
1,245455,3,2,1
2,245478,26,,15
3,245525,,,1
4,245527,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation
15,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation
45,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc."
47,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC"
48,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation
104,Sep 2 2022 8:26AM,245581,10,ISDIN Corporation
160,Sep 2 2022 8:18AM,245591,10,Emerginnova
163,Sep 2 2022 8:16AM,245590,10,ISDIN Corporation
164,Sep 2 2022 8:09AM,245588,10,ISDIN Corporation
180,Sep 2 2022 8:06AM,245587,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation,,,15
1,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation,,,30
2,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc.",1,,1
3,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC",,,1
4,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation,,,56
5,Sep 2 2022 8:26AM,245581,10,ISDIN Corporation,,,56
6,Sep 2 2022 8:18AM,245591,10,Emerginnova,,,3
7,Sep 2 2022 8:16AM,245590,10,ISDIN Corporation,,,1
8,Sep 2 2022 8:09AM,245588,10,ISDIN Corporation,,,16
9,Sep 2 2022 8:06AM,245587,19,"AdvaGen Pharma, Ltd",,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation,15,,
1,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation,30,,
2,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc.",1,,1
3,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation,56,,
5,Sep 2 2022 8:26AM,245581,10,ISDIN Corporation,56,,
6,Sep 2 2022 8:18AM,245591,10,Emerginnova,3,,
7,Sep 2 2022 8:16AM,245590,10,ISDIN Corporation,1,,
8,Sep 2 2022 8:09AM,245588,10,ISDIN Corporation,16,,
9,Sep 2 2022 8:06AM,245587,19,"AdvaGen Pharma, Ltd",4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation,15,,
1,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation,30,,
2,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc.",1,,1
3,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation,56,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation,15.0,NaN,NaN
1,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation,30.0,NaN,NaN
2,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc.",1.0,NaN,1.0
3,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation,56.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 8:32AM,245593,10,ISDIN Corporation,15.0,0.0,0.0
1,Sep 2 2022 8:30AM,245592,10,ISDIN Corporation,30.0,0.0,0.0
2,Sep 2 2022 8:30AM,245569,10,"Citieffe, Inc.",1.0,0.0,1.0
3,Sep 2 2022 8:30AM,245574,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Sep 2 2022 8:26AM,245576,10,ISDIN Corporation,56.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3192416,190.0,0.0,1.0
102,245555,0.0,1.0,0.0
12,491161,5.0,0.0,0.0
15,245478,15.0,0.0,26.0
16,982244,3.0,1.0,0.0
17,491165,2.0,0.0,0.0
19,245587,4.0,0.0,0.0
20,2455411,27.0,13.0,8.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3192416,190.0,0.0,1.0
1,102,245555,0.0,1.0,0.0
2,12,491161,5.0,0.0,0.0
3,15,245478,15.0,0.0,26.0
4,16,982244,3.0,1.0,0.0
5,17,491165,2.0,0.0,0.0
6,19,245587,4.0,0.0,0.0
7,20,2455411,27.0,13.0,8.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,190.0,0.0,1.0
1,102,0.0,1.0,0.0
2,12,5.0,0.0,0.0
3,15,15.0,0.0,26.0
4,16,3.0,1.0,0.0
5,17,2.0,0.0,0.0
6,19,4.0,0.0,0.0
7,20,27.0,13.0,8.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,190.0
1,102,Released,0.0
2,12,Released,5.0
3,15,Released,15.0
4,16,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20
Status,,,,,,,,
Completed,1.0,0.0,0.0,26.0,0.0,0.0,0.0,8.0
Executing,0.0,1.0,0.0,0.0,1.0,0.0,0.0,13.0
Released,190.0,0.0,5.0,15.0,3.0,2.0,4.0,27.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20
0,Completed,1.0,0.0,0.0,26.0,0.0,0.0,0.0,8.0
1,Executing,0.0,1.0,0.0,0.0,1.0,0.0,0.0,13.0
2,Released,190.0,0.0,5.0,15.0,3.0,2.0,4.0,27.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20
0,Completed,1.0,0.0,0.0,26.0,0.0,0.0,0.0,8.0
1,Executing,0.0,1.0,0.0,0.0,1.0,0.0,0.0,13.0
2,Released,190.0,0.0,5.0,15.0,3.0,2.0,4.0,27.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()